## Oppstart

In [1]:
from dhlab.module_update import update, css

In [2]:
css()

In [3]:
import warnings
warnings.filterwarnings('ignore')

### Note that the file `nbtext.py` will be overwritten in the code below

This is probably ok, though. If not, change `update('nbtext', overwrite = True)` to `update('nbtext', overwrite = False)` and the file will stay unchanged. 


In [4]:
import dhlab.nbtext as nb
import dhlab.token_map as tm
from dhlab.nbtext import get_urn, make_graph, get_urn, relaterte_ord, navn, totals
import dhlab.graph_networkx_louvain as gnl
import pandas as pd
%matplotlib inline

plot = lambda x,  lw=3, alpha= 0.7: x.plot(figsize=(15,6), lw=lw, alpha=alpha)


# Hjelpekommandoer

`alle_korpusnavn` henter ut alle navn (eller forslag til) fra korpuset.
`tell_navn` teller opp navnene på tvers av korpuset 


In [ ]:
import pandas as pd

def alle_korpusnavn(korpus):
    urner = list(korpus['urn'])
    alle_navn = tm.combine_names(tm.corpus_names(urner))
    return alle_navn

def tell_navn(korpus, token_map):
    res = dict()
    for urn in korpus['urn']:
        res[urn] = tm.count_name_strings(str(urn), token_map).to_dict()[0]
    return pd.DataFrame(pd.DataFrame(res).sum(axis=1).sort_values(ascending=False))

## Navn i bøker


Vi søker etter navn i en bok, og en serie av bøker.

Kommandoene er:
1. `names` henter et sett med forslag til navn, fra nbtext
1. `show_names` gir en oversikt over funnene, fra token_map
1. `names_to_token_map_file` lagrer navneforslagene til en excelfil (eller csv) for redigering (token_map)
1. `read_token_map_file` henter redigert fil tilbake for analyse (token_map)
1. `character_network` lager graf for navnene (token_map)
1. `show_graph`  tegner grafen (fra graph_networkx_louvain)
1. `show_communities` viser clustre i grafen (graph_networkx_louvain)


# Eksempel

Henter ut navn fra et korpus. Bøkene slås sammen.

## Korpus

Bygg først et korpus, enten fra kommando `nb.book_corpus` eller `book_urn`, eller gjenskap fra fil.

In [ ]:
korpus = nb.book_corpus(author='hamsun%')

In [ ]:
#korpus = nb.restore_metadata_from_excel('metadatafil.xls')

## Sjekk innholdet

Gjør noen søk for å se at alt er ok. Konkordanser er fine til det. Her går urnene gjennom `nb.pure_urn` -  i fremtiden skal det ikke være nødvendig.

In [ ]:
nb.urn_concordance(word = 'arbeid', urns = nb.pure_urn(korpus))

## Navn

Finn forslag til navn med `names`. Se på dem med `show_names`. Her er det muligheter til å skru litt på parametrene. Hvis ratioen skrus ned fjernes en del ord, og om cutoff settes høyere vil også enkelte navneforslag kunne forsvinne.

Velg hvilken bok som skal studeres (alle kan tas, men se under). Se på listen over korpus og sett den indeksen som passer. Her er det bare å føle seg frem.

In [ ]:
korpus

Finn korpuset ved å sette inn en index tilsvarende den som står helt til venstre i listen over.

Hent ut navn fra boken

In [ ]:
total_names = alle_korpusnavn(korpus)

# Ta en titt på navneforslagene

Men redigere dem i excel for å se. Navnene er hentet fra hele korpuset.

In [ ]:
total_names

### Lagring og redigering

Rediger filen fra cellen med lagringskommando (`names_to_token_map_file(....)`)
Listen trenger litt redigering, og sammenslåing. Lagre til Excel, og benytt `orient = column` for å ha navn i kolonnner, eller `orient = row` for å redigere radvis. Noen store filer med over 256 navn krever `row`. Kommandoen sjekker om filen allerede fins, og vil ikke skrive over. Følg med på output.

In [ ]:
tm.names_to_token_map_file(total_names, filename = "korpus_test0.xls", orient = "row")

### Redigeringsaktivitet

Åpne i for eksempel i excel eller tilsvarende som for eksempel openoffice eller libreoffice. Etter redigering, kjøres cellen under. Husk bare å benytte riktig filnavn. 

Husk at verdien på orient må være lik mellom lagring og lesing av excelfil.

In [ ]:
redigerte_navn = tm.read_token_map_file("korpus_test0.xls", orient = "row")

## Tell opp navn med hensyn til det som står i `redigerte_navn`

Bruker hjelpekommando `tell_navn`

In [ ]:
opptelling = tell_navn(korpus, redigerte_navn)
opptelling

# Lag en graf for en bok

Grafene kan legges sammen i etterkant om det ønskes en graf for hele korpuset.

In [ ]:
# fyll inn en indeks fra korpuslisten 0 er første URN, 1 neste osv.

G = tm.character_network(str(korpus['urn'][0]), redigerte_navn)
gnl.show_graph(G)

Dernest kan den vises og analyseres med passende kommandoer